<cell_type>markdown</cell_type># linkinthe.video - Open Source Pipeline

**Modeller:**
- Whisper - Transcription (fallback)
- Qwen2 7B - Ürün çıkarma
- LLaVA 1.6 - Frame analizi

**Akış:**
1. Transcript al (YouTube altyazısı veya Whisper)
2. LLM ile ürün çıkar → found[] + lost[]
3. Lost varsa → Video indir → Frame çıkar → Vision ile tanı

**Lazy Evaluation:** Video sadece gerektiğinde indirilir.

---

<cell_type>markdown</cell_type>## 0. Kurulum

**ÖNEMLİ:** Runtime → Change runtime type → T4 GPU seç!

In [ ]:
# GPU kontrol
!nvidia-smi

import torch
if torch.cuda.is_available():
    print(f"\n✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("❌ GPU YOK! Runtime → Change runtime type → T4 GPU seç!")

In [ ]:
# Paketleri kur
!pip install -q youtube-transcript-api openai-whisper
!pip install -q transformers accelerate bitsandbytes sentencepiece protobuf

print("✅ Kurulum tamamlandı!")

In [ ]:
import os
import json
import time
import requests

# Çalışma klasörü
WORK_DIR = "/content/linkinthe_test"
os.makedirs(WORK_DIR, exist_ok=True)

# Test videosu
VIDEO_URL = "https://www.youtube.com/watch?v=WlgjElhWD-U"
VIDEO_ID = VIDEO_URL.split("v=")[-1].split("&")[0]

# State
local_video_path = None
transcript_segments = None  # timestamp'li
transcript_text = None

print(f"Video ID: {VIDEO_ID}")
print(f"Video URL: {VIDEO_URL}")

<cell_type>markdown</cell_type>---
## 1. Transcript Al

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

# Önce YouTube altyazısını dene
try:
    transcript_segments = YouTubeTranscriptApi.get_transcript(VIDEO_ID, languages=['tr', 'en'])
    transcript_text = " ".join([s['text'] for s in transcript_segments])
    print(f"✅ YouTube altyazısı bulundu! ({len(transcript_text)} karakter)")
    print(f"   {len(transcript_segments)} segment (timestamp'li)")
    print("\n" + "="*60)
    print(transcript_text[:1000])
    print("="*60)
except Exception as e:
    print(f"❌ YouTube altyazısı yok: {e}")
    print("→ Video indirip Whisper kullanacağız...")

<cell_type>markdown</cell_type>---
## 2. Video Yükle (sadece altyazı yoksa)

YouTube altyazısı varsa bu adımı **ATLA**.

In [ ]:
# Manuel video upload
from google.colab import files

local_video_path = f"{WORK_DIR}/video.mp4"

print("📁 Video dosyasını yükle (.mp4)")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    with open(local_video_path, 'wb') as f:
        f.write(uploaded[filename])
    size_mb = os.path.getsize(local_video_path) / (1024 * 1024)
    print(f"✅ Video yüklendi: {size_mb:.1f} MB")

In [ ]:
# Transcript yoksa → indir + Whisper
if not transcript_text:
    local_video_path = download_video(VIDEO_URL, VIDEO_ID)
    
    if local_video_path:
        # Ses çıkar
        audio_path = f"{WORK_DIR}/audio.wav"
        !ffmpeg -i "{local_video_path}" -vn -acodec pcm_s16le -ar 16000 -ac 1 "{audio_path}" -y -loglevel error
        print(f"✅ Ses çıkarıldı")
        
        # Whisper
        import whisper
        print("Whisper yükleniyor...")
        model = whisper.load_model("medium")
        
        print("Transkript alınıyor...")
        result = model.transcribe(audio_path)
        
        transcript_text = result['text']
        transcript_segments = result['segments']
        print(f"✅ Whisper tamamlandı! ({len(transcript_text)} karakter)")
        
        del model
        torch.cuda.empty_cache()
else:
    print("⏭️ YouTube altyazısı var, video indirmeye gerek yok.")

<cell_type>markdown</cell_type>---
## 3. LLM ile Ürün Çıkar (found/lost)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Qwen2 7B - tamamen açık, çok iyi performans
MODEL_ID = "Qwen/Qwen2-7B-Instruct"

print("Qwen2 7B yükleniyor...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
llm_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)
print("✅ Qwen2 yüklendi!")

In [ ]:
def ask_llama(prompt, max_tokens=1500):
    messages = [
        {"role": "system", "content": "Sen bir ürün tespit asistanısın. JSON formatında cevap ver."},
        {"role": "user", "content": prompt}
    ]
    input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(llm_model.device)
    outputs = llm_model.generate(input_ids, max_new_tokens=max_tokens, do_sample=True, temperature=0.1, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)

# Ürün çıkarma prompt'u - found/lost ayırıyor
PRODUCT_PROMPT = f"""Bu bir YouTube videosunun transkripti. Videoda bahsedilen ürünleri bul.

KURALLAR:
1. Emin olduğun ürünler (marka+model belli) → "found" listesine
2. Belirsiz olanlar (sadece "kablo", "tripod" gibi) → "lost" listesine  
3. Duplicate ekleme, aynı ürünü bir kere yaz
4. Her ürün için timestamp (saniye) ver

JSON formatı:
{{
  "found": [{{"name": "Sony A7IV", "category": "kamera", "timestamp": 125}}],
  "lost": [{{"name": "kablo", "category": "aksesuar", "timestamp": 340}}]
}}

Transkript:
---
{transcript_text[:6000]}
---

SADECE JSON dön."""

print("Ürünler çıkarılıyor...")
start = time.time()
llm_response = ask_llama(PRODUCT_PROMPT)
print(f"✅ Tamamlandı! ({time.time()-start:.1f} sn)")
print(llm_response)

In [ ]:
# JSON parse et
def parse_llm_json(text):
    clean = text
    if "```json" in clean:
        clean = clean.split("```json")[1].split("```")[0]
    elif "```" in clean:
        clean = clean.split("```")[1].split("```")[0]
    start_idx = clean.find("{")
    end_idx = clean.rfind("}") + 1
    if start_idx != -1 and end_idx > start_idx:
        clean = clean[start_idx:end_idx]
    return json.loads(clean)

try:
    data = parse_llm_json(llm_response)
    found = data.get("found", [])
    lost = data.get("lost", [])
    
    print(f"✅ FOUND ({len(found)} ürün):")
    for p in found:
        print(f"   🟢 {p['name']} ({p.get('category', '?')}) @ {p.get('timestamp', '?')}s")
    
    print(f"\n⚠️ LOST ({len(lost)} ürün) - Vision ile çözülecek:")
    for p in lost:
        print(f"   🟡 {p['name']} ({p.get('category', '?')}) @ {p.get('timestamp', '?')}s")
except Exception as e:
    print(f"❌ Parse hatası: {e}")
    found, lost = [], []

# Llama'yı temizle
del llm_model, tokenizer
torch.cuda.empty_cache()
print("\n✅ Llama bellekten silindi.")

In [ ]:
<cell_type>markdown</cell_type>---
## 4. Lost Ürünleri Vision ile Çöz

Video sadece lost varsa indirilir.

In [ ]:
if not lost:
    print("✅ Lost yok, Vision'a gerek yok!")
    vision_model = None
else:
    # Video indir (henüz indirilmediyse)
    if not local_video_path:
        print("Lost var, video indiriliyor...")
        local_video_path = download_video(VIDEO_URL, VIDEO_ID)
    
    if local_video_path:
        # LLaVA yükle
        from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
        from PIL import Image as PILImage
        
        print("LLaVA yükleniyor...")
        VISION_MODEL = "llava-hf/llava-v1.6-mistral-7b-hf"
        vision_processor = LlavaNextProcessor.from_pretrained(VISION_MODEL)
        vision_model = LlavaNextForConditionalGeneration.from_pretrained(
            VISION_MODEL,
            torch_dtype=torch.float16,
            device_map="auto",
            load_in_4bit=True,
        )
        print("✅ LLaVA yüklendi!")
    else:
        print("❌ Video indirilemedi, Vision atlanıyor")
        vision_model = None

In [ ]:
def extract_frame(video_path, timestamp_sec):
    """Video'dan belirli saniyede frame çıkar."""
    frame_path = f"{WORK_DIR}/frame_{timestamp_sec}.jpg"
    !ffmpeg -ss {timestamp_sec} -i "{video_path}" -vframes 1 -q:v 2 "{frame_path}" -y -loglevel error
    return frame_path if os.path.exists(frame_path) else None

def analyze_frame(image_path, product_hint):
    """LLaVA ile frame analiz et."""
    image = PILImage.open(image_path)
    prompt = f"Bu frame'de '{product_hint}' olarak bahsedilen ürünü tanıyabilir misin? Marka ve model adını söyle. Tanıyamıyorsan 'UNKNOWN' de."
    
    conversation = [{"role": "user", "content": [{"type": "image"}, {"type": "text", "text": prompt}]}]
    formatted = vision_processor.apply_chat_template(conversation, add_generation_prompt=True)
    inputs = vision_processor(images=image, text=formatted, return_tensors="pt").to(vision_model.device)
    output = vision_model.generate(**inputs, max_new_tokens=100, do_sample=False)
    result = vision_processor.decode(output[0], skip_special_tokens=True)
    if "[/INST]" in result:
        result = result.split("[/INST]")[-1].strip()
    return result

# Lost ürünleri çöz
if lost and vision_model:
    print(f"\n{len(lost)} lost ürün Vision ile analiz ediliyor...\n")
    resolved = []
    
    for product in lost:
        ts = product.get("timestamp", 0)
        name = product.get("name", "ürün")
        print(f"🔍 '{name}' @ {ts}s ...", end=" ")
        
        frame_path = extract_frame(local_video_path, ts)
        if frame_path:
            guess = analyze_frame(frame_path, name)
            print(f"→ {guess[:50]}")
            
            if guess and "UNKNOWN" not in guess.upper():
                product["name"] = guess.split("\n")[0][:100]  # İlk satır, max 100 char
                product["resolved_by"] = "vision"
                found.append(product)
                resolved.append(product)
    
    # Resolved olanları lost'tan çıkar
    for r in resolved:
        if r in lost:
            lost.remove(r)
    
    print(f"\n✅ {len(resolved)} ürün Vision ile çözüldü!")

In [ ]:
<cell_type>markdown</cell_type>---
## 5. Sonuçlar

In [ ]:
print("="*60)
print("SONUÇLAR")
print("="*60)

print(f"\n✅ FOUND ({len(found)} ürün):")
for i, p in enumerate(found, 1):
    resolved = " [Vision]" if p.get("resolved_by") == "vision" else ""
    print(f"   {i}. {p['name']} ({p.get('category', '?')}){resolved}")

print(f"\n❌ LOST ({len(lost)} ürün) - Tanımlanamadı:")
for i, p in enumerate(lost, 1):
    print(f"   {i}. {p['name']} ({p.get('category', '?')})")

print("\n" + "="*60)
print(f"Video indirildi mi: {'Evet' if local_video_path else 'Hayır'}")
print(f"Transcript kaynağı: {'YouTube' if not local_video_path else 'Whisper'}")
print("="*60)

---
## 6. LLaVA ile Frame Analizi

LLaVA - Open source vision-language model.

In [ ]:
def analyze_frame_with_llava(image_path, prompt):
    """LLaVA ile frame analiz et."""
    image = PILImage.open(image_path)
    
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt},
            ],
        },
    ]
    
    formatted = vision_processor.apply_chat_template(
        conversation, add_generation_prompt=True
    )
    
    inputs = vision_processor(
        images=image,
        text=formatted,
        return_tensors="pt"
    ).to(vision_model.device)
    
    output = vision_model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=False,
    )
    
    result = vision_processor.decode(output[0], skip_special_tokens=True)
    # Sadece cevabı al
    if "[/INST]" in result:
        result = result.split("[/INST]")[-1].strip()
    return result

# Test
if frames:
    print("Test frame analizi...")
    display(Image(filename=frames[0], width=300))
    
    test_result = analyze_frame_with_llava(
        frames[0],
        "What products or devices do you see in this image? List them briefly."
    )
    print(f"\nSonuç: {test_result}")

---
## 7. Sonuçları Birleştir

---
## 8. Performans ve Maliyet Özeti

---
## Notlar ve Değerlendirme

### Kalite Değerlendirmesi

| Model | Kalite (/10) | Notlar |
|-------|--------------|--------|
| Whisper | /10 | |
| Llama 3.1 | /10 | |
| LLaVA | /10 | |

### Bulunan Ürünler Doğru mu?
- [ ] Evet, çoğu doğru
- [ ] Yarı yarıya
- [ ] Çoğu yanlış

### Kaçan Ürünler:
- ...

### Yanlış Tespitler:
- ...

### API vs Open Source:
- [ ] Open source yeterli
- [ ] API daha iyi ama open source kabul edilebilir
- [ ] API şart, open source yetersiz

### Sonraki Adımlar:
- ...